In [1]:
!pip install keras-preprocessing



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.8 MB/s eta 0:00:00


In [2]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
TRAIN_DIR = '/content/drive/MyDrive/imagesOld/train'
TEST_DIR = '/content/drive/MyDrive/imagesOld/test'

In [5]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels

In [6]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)
print(train)

neutral completed
fear completed
angry completed
happy completed
surprise completed
sad completed
                                                  image    label
0     /content/drive/MyDrive/imagesOld/train/neutral...  neutral
1     /content/drive/MyDrive/imagesOld/train/neutral...  neutral
2     /content/drive/MyDrive/imagesOld/train/neutral...  neutral
3     /content/drive/MyDrive/imagesOld/train/neutral...  neutral
4     /content/drive/MyDrive/imagesOld/train/neutral...  neutral
...                                                 ...      ...
1990  /content/drive/MyDrive/imagesOld/train/sad/929...      sad
1991  /content/drive/MyDrive/imagesOld/train/sad/934...      sad
1992  /content/drive/MyDrive/imagesOld/train/sad/953...      sad
1993  /content/drive/MyDrive/imagesOld/train/sad/988...      sad
1994  /content/drive/MyDrive/imagesOld/train/sad/933...      sad

[1995 rows x 2 columns]


In [7]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)
print(test)

happy completed
neutral completed
sad completed
surprise completed
fear completed
angry completed
                                                 image  label
0    /content/drive/MyDrive/imagesOld/test/happy/11...  happy
1    /content/drive/MyDrive/imagesOld/test/happy/12...  happy
2    /content/drive/MyDrive/imagesOld/test/happy/11...  happy
3    /content/drive/MyDrive/imagesOld/test/happy/11...  happy
4    /content/drive/MyDrive/imagesOld/test/happy/11...  happy
..                                                 ...    ...
624  /content/drive/MyDrive/imagesOld/test/angry/76...  angry
625  /content/drive/MyDrive/imagesOld/test/angry/96...  angry
626  /content/drive/MyDrive/imagesOld/test/angry/81...  angry
627  /content/drive/MyDrive/imagesOld/test/angry/86...  angry
628  /content/drive/MyDrive/imagesOld/test/angry/84...  angry

[629 rows x 2 columns]


In [8]:
from tqdm.notebook import tqdm

# Data Preprocessing

In [9]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        try:
            # Load the image
            img = load_img(image, color_mode='grayscale')
            # Convert to numpy array
            img = np.array(img)
            features.append(img)
        except Exception as e:
            print(f"Error processing {image}: {e}")
    return features

# Feature Extraction

In [11]:
train_features = extract_features(train['image'])
test_features = extract_features(test['image'])

  0%|          | 0/1995 [00:00<?, ?it/s]

  0%|          | 0/629 [00:00<?, ?it/s]

# Normalization

In [12]:
'''
x_train = train_features / 255.0
x_test = test_features / 255.0
'''

'\nx_train = train_features / 255.0\nx_test = test_features / 255.0\n'

In [13]:

import numpy as np

# Convert lists to NumPy arrays
train_features = np.array(train_features)
test_features = np.array(test_features)

# Perform element-wise division
x_train = train_features / 255.0
x_test = test_features / 255.0

'''
# Reshape and normalize input data
x_train = x_train.reshape((-1, 48, 48, 1)).astype('float32') / 255.0
x_test = x_test.reshape((-1, 48, 48, 1)).astype('float32') / 255.0
'''

"\n# Reshape and normalize input data\nx_train = x_train.reshape((-1, 48, 48, 1)).astype('float32') / 255.0\nx_test = x_test.reshape((-1, 48, 48, 1)).astype('float32') / 255.0\n"

In [14]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train['label'])
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

In [15]:

print(len(x_train))  # Should print 629
print(len(y_train))  # Should print 629


1995
1995


In [16]:

if len(x_train) > len(y_train):
    x_train = x_train[:len(y_train)]
elif len(y_train) > len(x_train):
    y_train = y_train[:len(x_train)]


In [17]:

# If x_train has fewer samples, you can truncate y_train
x_train = x_train[:len(y_train)]

# If y_train has fewer samples, you can truncate x_train
y_train = y_train[:len(x_train)]

In [18]:
from sklearn.utils import resample

# Repeat samples to match size
x_train_resampled, y_train_resampled = resample(x_train, y_train, replace=True, n_samples=len(x_train), random_state=42)


In [19]:
assert len(x_train) == len(y_train), "Mismatch between input and label data"


# Building the Neural Network Model

In [20]:
# sequential model
model = Sequential()

# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

# flattening
model.add(Flatten())

# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

# output layer
model.add(Dense(7, activation='softmax'))

'''
# model compilation
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy' )
'''
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Training the Model

In [21]:
model.fit(x=x_train, y=y_train, batch_size=128, epochs=200, validation_data=(x_test, y_test))

Epoch 1/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 26s 929ms/step - accuracy: 0.1877 - loss: 1.8896 - val_accuracy: 0.1590 - val_loss: 1.8393
Epoch 2/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 17s 53ms/step - accuracy: 0.2147 - loss: 1.8214 - val_accuracy: 0.2035 - val_loss: 1.8475
Epoch 3/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.2090 - loss: 1.8040 - val_accuracy: 0.1590 - val_loss: 1.8262
Epoch 4/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.1989 - loss: 1.8015 - val_accuracy: 0.1606 - val_loss: 1.8208
Epoch 5/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 0.2042 - loss: 1.7991 - val_accuracy: 0.2035 - val_loss: 1.8053
Epoch 6/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.2200 - loss: 1.7864 - val_accuracy: 0.2035 - val_loss: 1.8163
Epoch 7/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.2143 - loss: 1.7854 - val_accuracy: 0.2035 - val_loss: 1.8084
Epoch 8/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.2188 - loss: 1.7774 - val_accuracy:

# Saving the Model

In [22]:
model_json = model.to_json()
with open("emotiondetector.json", 'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")